In [9]:
import pandas as pd

df = pd.read_csv("Hechos_Transito/nuevo_acumulado_hechos_de_transito_2024.csv", encoding='utf-8')
df.head()

,fecha_evento,hora_evento,tipo_evento,fecha_captura,folio,latitud,longitud,punto_1,punto_2,colonia,...,clasificacion_de_la_vialidad,sentido_de_circulacion,dia,prioridad,origen,unidad_medica_de_apoyo,matricula_unidad_medica,trasladado_lesionados,personas_fallecidas,personas_lesionadas
0,01/01/2024,08:03,ATROPELLADO,12/02/2024,C5/20240101/02177,19.235238,-99.124109,LIC MARTINEZ DE CASTRO,PINA Y PALACIOS,SAN MATEO XALPA,...,VIA SECUNDARIA,S-N,lunes,MEDIA,RADIO,ERUM,MX146,SI,0,1
1,01/01/2024,14:56,ATROPELLADO,12/02/2024,C5/20240101/03566,19.413139,-99.141246,SIMON BOLIVAR,FERNANDO RAMIREZ,OBRERA,...,VIA SECUNDARIA,N-S,lunes,BAJA,RADIO,PC,03,NO,0,1
2,01/01/2024,21:11,ATROPELLADO,12/02/2024,C5/20240101/05341,19.32514,-99.158258,AV AZTECAS,REYNA IXTLIXOCHITL,AJUSCO,...,VIA PRIMARIA,NO-SP,lunes,BAJA,LLAMADA DEL 911,ERUM,MX036G2,NO,0,1
3,01/01/2024,22:38,ATROPELLADO,12/02/2024,C5/20240101/05779,19.488136,-99.126116,EJE 5 NTE,CALI,LINDAVISTA,...,EJE VIAL,SO-NP,lunes,BAJA,LLAMADA DEL 911,ERUM,MX167,NO,0,1
4,01/01/2024,00:34,CHOQUE,12/02/2024,C5/20240101/00182,19.271021,-99.000268,20 DE NOVIEMBRE,JAVIER MINA,SAN MATEO,...,VIA SECUNDARIA,SP-NO,lunes,MEDIA,LLAMADA DEL 911,ERUM,MX370,SI,0,1


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30655 entries, 0 to 30654
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   fecha_evento                  30655 non-null  object 
 1   hora_evento                   30655 non-null  object 
 2   tipo_evento                   30655 non-null  object 
 3   fecha_captura                 30655 non-null  object 
 4   folio                         30655 non-null  object 
 5   latitud                       30655 non-null  object 
 6   longitud                      30655 non-null  float64
 7   punto_1                       30655 non-null  object 
 8   punto_2                       30654 non-null  object 
 9   colonia                       30655 non-null  object 
 10  alcaldia                      30655 non-null  object 
 11  zona_vial                     30655 non-null  int64  
 12  sector                        30655 non-null  object 
 13  u

In [14]:
import pandas as pd
import re

# Columnas candidatas a tener texto sucio (excluyendo fechas y horas por ahora)
cols_texto = [
    'tipo_evento', 'punto_1', 'punto_2', 'colonia', 'alcaldia', 
    'sector', 'unidad_a_cargo', 'tipo_de_interseccion', 
    'clasificacion_de_la_vialidad', 'sentido_de_circulacion', 
    'prioridad', 'origen', 'trasladado_lesionados'
]

patron_error = r'[ÃÂâ€]'  # Patrón común de UTF-8 interpretado como Latin-1

print("--- ANÁLISIS DE CARACTERES SOSPECHOSOS ---")
encontrado = False

for col in cols_texto:
    # Filtrar valores únicos que contengan el patrón de error
    unique_vals = df[col].astype(str).unique()
    errores = [val for val in unique_vals if re.search(patron_error, val)]
    
    if errores:
        encontrado = True
        print(f"\nCOLUMNA: {col}")
        print(f"Total valores únicos sospechosos: {len(errores)}")
        print("Ejemplos:", errores[:10]) # Muestra los primeros 10 para no saturar

if not encontrado:
    print("No se detectaron patrones obvios de codificación rota (Ã, Â, etc.).")

--- ANÁLISIS DE CARACTERES SOSPECHOSOS ---

COLUMNA: colonia
Total valores únicos sospechosos: 1
Ejemplos: ['ZONA RUSTICA TLÃ?HUAC']

COLUMNA: origen
Total valores únicos sospechosos: 1
Ejemplos: ['BOTÃ“N DE AUXILIO']


In [15]:
# Intentar conversión a numérico y filtrar los fallos
# 'coerce' convierte errores a NaN
latitud_numeric = pd.to_numeric(df['latitud'], errors='coerce')

# Filtrar filas donde la conversión falló (eran NaN originalmente o eran texto inválido)
# Se verifica si el original NO era nulo pero el numérico SI es nulo
filas_sucias = df[df['latitud'].notna() & latitud_numeric.isna()]

print("\n--- AUDITORÍA DE LATITUD (Type Object) ---")
if not filas_sucias.empty:
    print(f"Se encontraron {len(filas_sucias)} registros no numéricos en 'latitud'.")
    print("Valores únicos problemáticos:")
    print(filas_sucias['latitud'].unique())
else:
    print("Todos los valores parecen convertibles. Probablemente el tipo 'object' se debe a la carga inicial.")


--- AUDITORÍA DE LATITUD (Type Object) ---
Se encontraron 3 registros no numéricos en 'latitud'.
Valores únicos problemáticos:
['1123MP001' '19.398722 -99.172410' '19.315803,']


In [16]:
import pandas as pd
import numpy as np

# 1. Reparación de Mojibake (UTF-8 mal interpretado)
# Se aplica a las columnas donde detectaste errores y otras de texto relevantes
cols_texto = ['colonia', 'origen', 'tipo_evento', 'alcaldia', 'punto_1', 'punto_2']

def reparar_utf8(texto):
    if pd.isna(texto):
        return texto
    try:
        # Invierte la lectura incorrecta: Str(Win1252) -> Bytes -> Str(UTF-8)
        return texto.encode('cp1252').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        return texto

print("Aplicando corrección de codificación...")
for col in cols_texto:
    if col in df.columns:
        df[col] = df[col].apply(reparar_utf8)

# 2. Corrección de Latitud (Object -> Float)
# 'coerce' transformará cualquier string no numérico (ej. "null", error de dedo) en NaN
df['latitud'] = pd.to_numeric(df['latitud'], errors='coerce')

# 3. Verificación de resultados
print("-" * 30)
print("VERIFICACIÓN DE CORRECCIONES")
print(f"Latitud Dtype: {df['latitud'].dtype} (Esperado: float64)")
print(f"Nulos en Latitud: {df['latitud'].isna().sum()}")

# Muestra valores corregidos específicos
print("\nEjemplos corregidos (Colonia / Origen):")
muestras = df[df['colonia'].str.contains('TLAHUAC', case=False, na=False) | 
              df['origen'].str.contains('BOTON', case=False, na=False)]

if not muestras.empty:
    print(muestras[['colonia', 'origen']].head(5))
else:
    print(df[['colonia', 'origen']].head(5))
    

Aplicando corrección de codificación...
------------------------------
VERIFICACIÓN DE CORRECCIONES
Latitud Dtype: float64 (Esperado: float64)
Nulos en Latitud: 3

Ejemplos corregidos (Colonia / Origen):
                   colonia           origen
96    ZONA RUSTICA TLAHUAC  LLAMADA DEL 911
498      UNIDAD CUITLAHUAC  LLAMADA DEL 911
533            IXTLAHUACAN  LLAMADA DEL 911
684   ZONA RUSTICA TLAHUAC  LLAMADA DEL 911
1259                OBRERA  MI CALLE(BOTON)


In [17]:
import pandas as pd
import numpy as np
import os

# Supuesto: df ya está cargado y con latitud/encoding corregido del paso anterior

# 1. Estandarización Temporal
# Convertir a datetime. 'errors=coerce' pondrá NaT si hay fechas inválidas
print("Estandarizando fechas...")
df['fecha_evento'] = pd.to_datetime(df['fecha_evento'], dayfirst=True, errors='coerce')

# 2. Normalización de Alcaldías (Para cruzar con Inviales/Pluviales)
print("Normalizando alcaldías...")
mapa_acentos = str.maketrans("ÁÉÍÓÚÑ", "AEIOUN")
def limpiar_texto(x):
    if pd.isna(x): return "DESCONOCIDO"
    return str(x).upper().translate(mapa_acentos).strip()

df['alcaldia'] = df['alcaldia'].apply(limpiar_texto)

# 3. Validación de Coordenadas (Filtro Geográfico Básico CDMX)
# Latitud ~19, Longitud ~-99. Detectar outliers extremos (ej. 0,0)
mask_outliers = (df['latitud'] < 18) | (df['latitud'] > 20) | \
                (df['longitud'] < -100) | (df['longitud'] > -98)
outliers = df[mask_outliers]
if not outliers.empty:
    print(f"ADVERTENCIA: Se detectaron {len(outliers)} registros con coordenadas fuera de rango CDMX.")
    # Opcional: df = df[~mask_outliers] # Descomentar para eliminar

# 4. Guardado Final
ruta_salida = r"Data_Tratado\Hechos_Transito.csv"
os.makedirs("Data_Tratado", exist_ok=True)

df.to_csv(ruta_salida, index=False, encoding='utf-8-sig')

print("-" * 30)
print(f"ARCHIVO GUARDADO: {ruta_salida}")
print(f"Total registros: {len(df)}")
print(f"Rango de fechas: {df['fecha_evento'].min()} a {df['fecha_evento'].max()}")
print("Muestra de Alcaldías:", df['alcaldia'].unique()[:5])

Estandarizando fechas...
Normalizando alcaldías...
------------------------------
ARCHIVO GUARDADO: Data_Tratado\Hechos_Transito.csv
Total registros: 30655
Rango de fechas: 2024-01-01 00:00:00 a 2024-12-31 00:00:00
Muestra de Alcaldías: ['XOCHIMILCO' 'CUAUHTEMOC' 'COYOACAN' 'GUSTAVO A MADERO' 'TLAHUAC']
